# RadioML dataset - 2016.10A
----

- It is a synthetic dataset generated with GNU Radio.
- Consists of 11 modulations (8 digital and 3 analog) - E.g. - `8PSK`, `QPSK`, etc. Along with varying SNR ratios.
- [Source](https://www.deepsig.ai/datasets).

In [93]:
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
plt.rcParams["figure.figsize"] = (18,7)
plt.rcParams.update({'font.size': 15})
plt.style.use('fivethirtyeight')

In [275]:
# Import the pickled raw file
path = "/Volumes/DHIRAJ/B.Tech/Project-1/RML2016.10a_dict.pkl"
with open(path, 'rb') as pic_file:
    data = pickle.load(pic_file, encoding='latin1')

## About the dataset -
---

In [276]:
snr_vals, mod_classes = map(lambda j: sorted(list(set(map(lambda x: x[j], data.keys())))), [1,0])
print("Data is originally stored in a dictionary format with keys as (modulation, SNR value).")
print(f"Input Data shape: {data[('QAM16', 18)].shape}")
print(f"Total labels: {len(data.keys())}")
print(f"Modulation Techniques: {mod_classes}")
print(f"SNR values: {snr_vals}")

Data is originally stored in a dictionary format with keys as (modulation, SNR value).
Input Data shape: (1000, 2, 128)
Total labels: 220
Modulation Techniques: ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']
SNR values: [-20, -18, -16, -14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


## Exploratory Data analysis
---

### Different signal representations
---

In [277]:
def plot_data(samples, mod_name, SNR, plot_type="iq"):
    I_values = samples[0]
    Q_values = samples[1]
    signal = I_values + 1j * Q_values
    if plot_type == "iq":
        plt.plot(I_values, label="I values", c='r', linewidth=3, alpha=0.75)
        plt.plot(Q_values, label="Q values", linewidth=3, alpha=0.75)
        plt.plot(np.abs(signal), label="Amplitude", c='b', linewidth=4, alpha=0.85)
        plt.title(f"IQ/Amplitude Vs Time{mod_name}-{str(SNR)}")
        plt.xlabel("Time in $\mu s$")
        plt.legend()
        plt.show()
    if plot_type == "ph_amp":
        plt.subplot(1,2,1)
        plt.suptitle(f"Amplitude and Phase Vs Time ({mod_name}-{str(SNR)})")
        plt.plot(np.abs(signal), label="Amplitude", c='b', linewidth=3, alpha=0.75)
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(np.angle(signal), label="Phase", c='g', linewidth=3, alpha=0.75)
        plt.legend()
        plt.show()
    if plot_type == "spec":
        spec = plt.specgram(signal,  Fs=1000, mode='magnitude')
        plt.colorbar()
        plt.title(f"{mod_name}-{str(SNR)}")
        plt.show()
# d = plt.magnitude_spectrum(c, Fs=1000)
# plt.show()

In [ ]:
# plot_data(data[("QPSK", 10)][0], "QPSK", 10, "iq")
# plot_data(data[("QPSK", 10)][0], "QPSK", 10, "ph_amp")
# plot_data(data[("QPSK", 10)][0], "QPSK", 10, "spec")
# plot_data(data[("AM-SSB", 10)][0], "AM-SSB", 10, "iq")
# plot_data(data[("AM-SSB", 10)][0], "AM-SSB", 10, "ph_amp")

In [ ]:
# ex = data[("QAM16", 18)][0]
# i = ex[0]
# q = ex[1]
# c = i + 1j * q
# s = plt.specgram(c,  Fs=1000000, mode='psd')
# plt.colorbar()
# plt.show()
# plt.plot(s[1], s[0])
# plt.show()

In [ ]:
data[('BPSK',  4)][0][0]

# Data Preprocessing
---
Steps -

1. Remove the analog modulation techniques.
2. Separate the training and testing data - Using `70%` (700/1000) for training and `30%` for testing.
3. Create appropriate labels for the training step

In [279]:
analog_mods = ['AM-SSB', 'WBFM', 'QAM16', 'QAM64']
removed_keys = [key for key in data if key[0] in analog_mods]
digital_data = {key: data[key] for key in data if key not in removed_keys}
digital_snrs, digital_mods = map(lambda j: sorted(list(set(map(lambda x: x[j], digital_data.keys())))), [1,0])
print(f"Total digital (mods, SNR) pairs (labels): {len(digital_data.keys())}")
print(f"Digital Mods: {digital_mods}")

Total digital (mods, SNR) pairs (labels): 140
Digital Mods: ['8PSK', 'AM-DSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QPSK']


In [280]:
train_test_ratio = 0.6
train_dict = {}
test_dict = {}
for key, value in digital_data.items():
    train_dict.update({key: value[:int(value.shape[0] * train_test_ratio), :]})
    test_dict.update({key: value[int(value.shape[0] * train_test_ratio):, :]})
print(f"Training set dimensions: {train_dict[('8PSK',2)].shape}")
print(f"Testing set dimensions: {test_dict[('8PSK',2)].shape}")

Training set dimensions: (600, 2, 128)
Testing set dimensions: (400, 2, 128)


In [282]:
train_snrs, train_mods = map(lambda j: sorted(list(set(map(lambda x: x[j], train_dict.keys())))), [1,0])
X_train = []  
train_labels = []
for mod in train_mods:
    for snr in train_snrs:
        X_train.append(train_dict[(mod, snr)])
        for sample_number in range(train_dict[(mod,snr)].shape[0]):
            train_labels.append((mod,snr))
X_train = np.vstack(X_train)
n_samples_train, dim1, dim2 = X_train.shape 
y_train = np.array(list(map(lambda x: train_mods.index(train_labels[x][0]), range(n_samples_train))))

In [283]:
test_data = defaultdict(list)
test_labels = defaultdict(list)
def get_test_data_snr(snr):
    for key, val in test_dict.items():
        if key[1] == snr:
            test_data[snr].append(val)
            for x in range(val.shape[0]):
                # For every sample with that SNR value
                test_labels[snr].append(key[0]) # Save the mod name
    test_data[snr] = np.vstack(test_data[snr])
    n_samples_test = test_data[snr].shape[0]
    test_labels[snr] = np.array(list(map(lambda x: train_mods.index(test_labels[snr][x]), range(n_samples_test))))
    return test_data[snr], test_labels[snr]

X_test = defaultdict(list)
y_test = defaultdict(list)
for snr in train_snrs:
    data, labels = get_test_data_snr(snr)
    X_test[snr].append(data)
    X_test[snr] = np.vstack(X_test[snr])
    y_test[snr].append(labels)
    y_test[snr] = np.hstack(y_test[snr])
print(y_test[-20].shape)

(2800,)


## Feature Extraction
----


In [284]:
def compute_standard_deviation(A, B, count):
    sigma = np.power(A/count - np.power(B/count, 2), 0.5)
    return sigma

In [285]:
def extract_features(X):
    FS = 1000 # 1000kHz
    FC = 45 # 45kHz
    features = np.zeros([len(X), 8]) # Start with 8 features
    for ex_no in range(X.shape[0]):
        real_I = X[ex_no][0]
        imag_Q = X[ex_no][1]
        index_record = np.array([], dtype=int) # To hold the index for > threshold
        signal = real_I + imag_Q * 1j # Construct the signal
        total_frames = len(signal)
        # Amplitude computations
        A_i = np.power(np.power(real_I, 2) + np.power(imag_Q, 2), 0.5)
        A_n = A_i / np.mean(A_i)
        A_cn = A_n - 1
        # Phase computations
        threshold_a_cn = np.zeros(total_frames)
        phase = np.angle(signal)
        phi_unwrap = np.unwrap(phase)
        phi_NL = np.zeros(total_frames)
        # Subtract the linear part of the phase.
        for i in range(total_frames):
            if A_n[i] > 1:
                phi_NL[i] = phi_unwrap[i] - (2 * np.pi * FC * i / FS)
                index_record = np.append(index_record, i)
                threshold_a_cn[i] = A_cn[i]
            
        phase_squared = np.sum(np.power(phi_NL, 2))
        phase_abs = np.sum(np.abs(phi_NL))
        phase_sum = np.sum(phi_NL)
        thresh_acn_squared = np.sum(np.power(threshold_a_cn, 2))
        thresh_acn_sum = np.sum(threshold_a_cn)
        FN = np.zeros(len(index_record)-1)

        # Frequency computations
        for index, thresh_index in enumerate(index_record):
            if index < len(index_record)-1:
                FN[index] = (phi_NL[index_record[index+1]] - phi_NL[thresh_index]) / (2*np.pi)
        Fn_squared = np.sum(np.power(FN, 2))
        Fn_abs = np.sum(np.abs(FN))
        
        # Statistical features
        deviations = signal - np.mean(signal)

        #### Compute the features one by one
        """ 1. Max value of power spectral density (PSD). Represents the variations in Amp,
        differentiate between Amp and non-amp modulations.
        """
        gamma_max = np.max(np.power(np.abs(np.fft.fft(A_cn)), 2)) / total_frames
        sigma_aa = np.power(np.mean(np.power(A_cn, 2)) - np.power(np.mean(np.abs(A_cn)),2), 0.5)
        sigma_ap = compute_standard_deviation(phase_squared, phase_abs, len(index_record))
        sigma_dp = compute_standard_deviation(phase_squared, phase_sum, len(index_record))
        sigma_a = compute_standard_deviation(thresh_acn_squared, thresh_acn_sum, len(index_record))
        sigma_nf = compute_standard_deviation(Fn_squared, Fn_abs, len(index_record))
        Kurtosis = np.abs(np.mean(np.power(deviations, 4)) / np.power(np.mean(np.power(deviations, 2)), 2))
        Skewness = np.abs(np.mean(np.power(deviations, 3)) / np.power(np.mean(np.power(deviations, 2)), 1.5))
        
        #### Store the features
        features[ex_no][0] = gamma_max
        features[ex_no][1] = sigma_ap
        features[ex_no][2] = sigma_dp
        features[ex_no][3] = sigma_aa
        features[ex_no][4] = sigma_a
        features[ex_no][5] = sigma_nf
        features[ex_no][6] = Kurtosis
        features[ex_no][7] = Skewness
    return features

In [286]:
X_train_features = extract_features(X_train) # Send 10 examples for testing

In [287]:
X_test_features1 = extract_features(X_test[10])

# Model Building
---

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(X_train_features, y_train)

In [260]:
for snr in [-10,-2,2,4,10]:
    X_test_features = extract_features(X_test[snr])
    pred = svm.predict(X_test_features)
    acc = accuracy_score(y_test[snr], pred)
    print(acc)


0.338


# Model Evaluation
----


defaultdict(list,
            {-20: array([5, 5, 5, ..., 2, 2, 2]),
             -18: array([2, 2, 2, ..., 1, 1, 1]),
             -16: array([1, 1, 1, ..., 2, 2, 2]),
             -14: array([0, 0, 0, ..., 2, 2, 2]),
             -12: array([0, 0, 0, ..., 4, 4, 4]),
             -10: array([1, 1, 1, ..., 5, 5, 5]),
             -8: array([2, 2, 2, ..., 3, 3, 3]),
             -6: array([4, 4, 4, ..., 0, 0, 0]),
             -4: array([3, 3, 3, ..., 5, 5, 5]),
             -2: array([1, 1, 1, ..., 0, 0, 0]),
             0: array([2, 2, 2, ..., 5, 5, 5]),
             2: array([5, 5, 5, ..., 4, 4, 4]),
             4: array([1, 1, 1, ..., 2, 2, 2]),
             6: array([3, 3, 3, ..., 0, 0, 0]),
             8: array([4, 4, 4, ..., 3, 3, 3]),
             10: array([2, 2, 2, ..., 5, 5, 5]),
             12: array([0, 0, 0, ..., 4, 4, 4]),
             14: array([4, 4, 4, ..., 2, 2, 2]),
             16: array([1, 1, 1, ..., 3, 3, 3]),
             18: array([3, 3, 3, ..., 5, 5, 5])})

In [15]:
clf

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
# keys_list = list(data.keys())
# temp_list = []
# label_list = []
# for i in range(len(keys_list)):
#     curr_item = data[keys_list[i]] 
#     temp_list.append(curr_item)
#     for j in range(curr_item.shape[0]):
#         label_list.append(keys_list[i])
        
# # Convert all lists into numpy arrays.
# # X = np.array(temp_list).reshape(1200000,2,128)
# # Y = np.array(label_list)

In [ ]:
# snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], data.keys())))), [1,0])
# X_train = []  
# labels_train = []
# for mod in mods:
#     for snr in snrs:
#         X_train.append(data[(mod,snr)])
#         for i in range(data[(mod,snr)].shape[0]): 
#             labels_train.append((mod,snr))
# X_train = np.vstack(X_train)
# n_samples_train = X_train.shape[0]
# y_train = np.array(list(map(lambda x: mods.index(labels_train[x][0]), range(n_samples_train))))

# print("**Training data (raw data)**")
# print(X_train.shape,"training data, ", y_train.shape, "labels")